# Winter school - paramak trial
Most of this script method is based on:
 
Freidberg, J. P. et al. _“Designing a Tokamak Fusion reactor—How Does Plasma Physics
Fit In?”_ 

Physics of Plasmas 22, 7 (July 2015): 070901 © 2015 AIP Publishing


## Imports 

In [79]:
import paramak
import numpy as np

## Engineering and nuclear physics constraints  

In [80]:
# Electric plant 
P_electric = 1000e6 # [W]
thermal_efficiency = 0.4 # [-]

# Nuclear physics 
fast_neutrons_Li7_slowing_down_cross_section = 2e-28 # [barns to m2]
slow_neutrons_Li6_breeding_cross_section = 960e-28 # [barns to m2] - at 0.025eV
thermal_neutron_energy = 0.025 # [eV]
fast_neutron_energy = 14.1e6 # [eV]

# Heat flux 
neutron_wall_loading_limit = 4e6 # [W/m2]

# Magnetic 
max_TF_stress = 600e6 # [Pa]
max_TF_on_coil_field = 13 # [T]
max_coil_current_density = 20e6 # [Amp/m2]

max_power_recirculation = 0.10 # [-]

# Plasma heating 
wall_to_absorbed_RF_power_conversion_efficiency = 0.4 # [-]

# Plasma 
average_plasma_temperature = 14e3 # [eV]


## Plasma shape

In [81]:
# Aspect ratio
A = 4 # [-]

# Plasma elongation 
k = 1.7 # [-]

## Blanket thickness
"_In fact, it is not unreasonable to say that the
entire scale of a fusion reactor is largely determined by the
slowing down mean free path of 14.1 MeV neutrons in lithium_" 

"_The calculation below presents a simple model for determining the width of 
the combined slowing down and breeding sub-regions. These are the dominant contributions to the blanket size which is only 
slightly less than the overall width b_"

In [82]:
# Define Lithium number density 
Li7_number_density = 4.6e28 # [atoms/m3]
Li6_number_density = 0.34e28 # [atoms/m3]

# Calculate mean free path for neutrons slowing down by Li7 and for neutron tritium breeding in Li6 
neutron_slowing_down_mean_free_path = 1/(Li7_number_density*fast_neutrons_Li7_slowing_down_cross_section)
tritium_breeding_mean_free_path = 1/(Li6_number_density*slow_neutrons_Li6_breeding_cross_section)

# Output vs input neutron flux ratio
ratio_flux_in_wrt_out = 10e5 # [-] - Output flux/input flux

alpha_B = (tritium_breeding_mean_free_path/neutron_slowing_down_mean_free_path)*((fast_neutron_energy/thermal_neutron_energy)**0.5)
blanket_thickness = neutron_slowing_down_mean_free_path*np.log(1 + alpha_B*np.log(ratio_flux_in_wrt_out))
print('Resultant blanket thickness: ' + str(round(blanket_thickness, 2)) + 'm')

# Define thickness of auxiliary wall components, 
# e.g., vacuum vessel, first_wall, shield, neutron multiplier 
wall_shield_vacuum_vessel_thickness = 0.2 # [m]

b = blanket_thickness + wall_shield_vacuum_vessel_thickness

Resultant blanket thickness: 0.99m


## Major radius
We use the wall neutron power loading limit (MW/m2) and the assumed reactor 
electric power to calculate the major radius of the plasma by knowing that: 

$P_n = S*P_{lim-wall}$

We can calculate $P_{neutron}$ by knowing the total electrical power, the 
power conversion efficiency and the added energy from the exothermic tritium breeding 
reactions. 

The surface area is defined by: 

$S \approx 4\pi^2R_0a[(1 + k^2)^{0.5}]$ 

The results is that the major radius scales inversely with a but is independent of the magnetic field B0. 


In [113]:
# Total energy from neutrons, alpha particles and exothermic tritium production
# e.g., 22.6MeV = 14.1MeV + 3.5Mev + 4.8MeV 
energy_alpha = 3.5e6 # [eV]
energy_tritium_production = 4.8e6 # [eV]
energy_fusion = fast_neutron_energy + energy_alpha + energy_tritium_production # [eV] 

# Calculate major radius R0
R0 = (((1/(4*np.pi**2))*(fast_neutron_energy/energy_fusion)*(P_electric/(thermal_efficiency*neutron_wall_loading_limit))*((2/(1 + k**2))**(0.5)))*4)**0.5
print('Resultant major radius: ' + str(round(R0, 2)) + 'm')

Resultant major radius: 5.35m


## Central toroidal magnetic field
Getting the toroidal magnetic field at the centre of the plasma is straight forward
since it scales with radius from the peak field-on-coil: 

$B(R)/B_0 = R/R_0$ 

Where:

$B(R_0) = B_0 =$ peak field-on-coil (T) 

In [84]:
# Calculate the minor radius from the major radius and the aspect ratio
a = R0/A 

# Define the gap from the first wall to the plasma
plasma_wall_gap = 0.0 # 0.05 # [m] leave at 0m for now (simplified model)

# Calculate B0 at R0
B0 = max_TF_on_coil_field*(1 - (a + b + plasma_wall_gap)/R0)

print('Resultant central magnetic field: ' + str(round(B0, 2)) + ' Tesla')

Resultant central magnetic field: 6.88 Tesla


## Toroidal field coil thickness 
Our simplified toroidal field coils are made up only of superconducting magnet materials 
and mechanical support materials. The total radial coil thickness is therefor given by: 

$c = c_J + c_M $ 

The main forces affecting $C_M$ are the tensile and the centering forces. The main parameter affecting $C_J$ is the max current density limit. 

In [85]:
mu_0 = 1.256e-6 # [N/A2] - vacuum permeability 
e_B = (a + b)/R0 # ratio of distance from centre of plasma-to-coil to the major radius 

# Calculate mechanical support thickness 
a_M = ((B0**2)/(mu_0*max_TF_stress))*((2*e_B/(1+e_B)) + 0.5*np.log((1+e_B)/(1-e_B)))
mech_thickness = R0*(1 - e_B - ((1-e_B)**2 - a_M)**0.5)
print('Resultant radial thickness of mechanical supports: ' + str(round(mech_thickness, 2)) + 'm')

# Calcuclate conductor thickness
a_J = 2*B0/(mu_0*R0*max_coil_current_density)
conductor_thickness = R0*(1 - e_B - ((1-e_B)**2 - a_J)**0.5)
print('Resultant radial thickness of magnet conductor: ' + str(round(conductor_thickness, 2)) + 'm')

# Calculate total thickness 
total_thickness = conductor_thickness + mech_thickness
print('Resultant total magnet radial thickness: ' + str(round(total_thickness, 2)) + 'm')


Resultant radial thickness of mechanical supports: 0.4m
Resultant radial thickness of magnet conductor: 0.58m
Resultant total magnet radial thickness: 0.97m


## Number of toroidal field coils 
We can easily calculate the number of toroidal field coils by know the limit of 
the toroidal field ripple.  

In [87]:
max_tf_field_ripple = 0.02 # [-] 

tf_return_inner_radius = R0 + a + b
number_tf_coils = np.log(max_tf_field_ripple)/np.log((R0+a)/tf_return_inner_radius)
print('Estimated number of toroidal field coils: ' + str(int(number_tf_coils)))

Estimated number of toroidal field coils: 24


## Average plasma temperature 
The plasma temperature is chosen as to maximuse the fusion power density. The temperature profile is fairly flat at the peak power density so we can choose a fitting temperature around the peak. 

This is the volume-average temperature. 

In [90]:
plasma_temperature = 14e3 # [eV]
print('The required volume-averaged plasma temperature is: ' + str(round(plasma_temperature, 2)) + 'eV')

The required plasma temperature is: 14000.0eV


## Average plasma pressure
We can get the required average plasma pressure again to maximise the fusion power. 

In [112]:
plasma_pressure = 1.01e5*(8.76/(a**0.5)) # [Pa]
print('The required volume-averaged plasma pressure is: ' + str(round(plasma_pressure, 2)) + 'Pa')

# Calculate beta - the ration of the plasma pressure to the magnetic pressure. 
# This is a good indicator of the economic situation. 
beta = 100*(2*mu_0*(plasma_pressure))/(B0**2)
print('Resultant beta: ' + str(round(beta, 2)) + '%')


The required volume-averaged plasma pressure is: 761861.89Pa
Resultant beta: 4.05%


## Average plasma density 
The volume-average plasma density can be derived from its relationship to the plasma pressure and temperature. 
 

In [108]:
plasma_density = 1e20*1.66/(a**0.5)
print('The required volume-averaged plasma density is: ' + str(round(plasma_density, 2)) + '/m3')


The required volume-averaged plasma density is: 1.429416722280494e+20/m3


## Energy confinement time 
The energy confinement time is determined by the requirement that in steady 
state the thermal conduction losses are balanced by alpha particle heating. 

The plasma is assumed to be ignited.

In [129]:
confinement_time = 3*(np.pi**2)*R0*(a**2)*k*plasma_pressure*((energy_fusion*thermal_efficiency)/(energy_alpha*P_electric))
print('Required energy confinement time: ' + str(round(confinement_time, 2)) + 's')

Required energy confinement time: 0.95s


## Plasma current 
Calculating the plasma current involves some plasma physics for the first time in this project. 

We equate the required energy confinement factor to the empirically determined expression for confinement time, which then lets us solve for the plasma current. 

In [130]:
H = 1  # [-] H-mode enhancement factor 
am = 2.5 # [-] atomic mass number

Ip = (7.98*(confinement_time**1.08))/((H**1.08)*(R0**1.49)*(a**0.62)*(k**0.84)*((plasma_pressure)**0.44)*(B0**0.16)*(am**0.20))*(((energy_alpha*P_electric*1e-6)/(energy_fusion*thermal_efficiency))**0.74)
print('Required plasma current: ' + str(round(Ip, 2)) + 'MA')
### WRONG

Required plasma current: 0.04MA


## CAD model generation - Paramak

In [ ]:
ball_reactor = paramak.BallReactor(
   inner_bore_radial_thickness = 50,
   inboard_tf_leg_radial_thickness = 50,
   center_column_shield_radial_thickness= 50,
   divertor_radial_thickness = 100,
   inner_plasma_gap_radial_thickness = 50,
   plasma_radial_thickness = 200,
   outer_plasma_gap_radial_thickness = 50,
   firstwall_radial_thickness = 50,
   blanket_radial_thickness = 100,
   blanket_rear_wall_radial_thickness = 50,
   elongation = 2,
   triangularity = 0.55,
   rotation_angle = 180
)

ball_reactor.export_html_3d('ball_reactor.html')

In [ ]:
demo = paramak.EuDemoFrom2015PaperDiagram(rotation_angle=90)
demo.export_html_3d('demo_reactor.html')


## Heat flux 
Use simple scaling law



# 